# Evaluating Models

This notebook contains the code used for evaluating the following learning models:

-  **Standard GBDT** (_baseline 1_)
-  **Adversarial Boosting** (_baseline 2_)
-  **Non-Interferent GBDT** (our proposal)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json
import glob
import pickle
import dill
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm
import functools
from os import listdir
from os.path import isfile, join
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix
from nilib import *
from robust_forest import *

2019-05-27 11:32:12,466 *** INFO [robust_forest.py:1151 - __init__()] *** ***** Robust Decision Tree successfully created *****
2019-05-27 11:32:12,468 *** INFO [robust_forest.py:1152 - __init__()] *** *	Tree ID: 0
2019-05-27 11:32:12,470 *** INFO [robust_forest.py:1153 - __init__()] *** *	Attacker: <robust_forest.Attacker object at 0x7f85590785f8>
2019-05-27 11:32:12,471 *** INFO [robust_forest.py:1155 - __init__()] *** *	Splitting criterion: SSE
2019-05-27 11:32:12,472 *** INFO [robust_forest.py:1156 - __init__()] *** *	Max depth: 8
2019-05-27 11:32:12,473 *** INFO [robust_forest.py:1158 - __init__()] *** *	Min instances per tree node: 20
2019-05-27 11:32:12,474 *** INFO [robust_forest.py:1160 - __init__()] *** *	Max samples: 100.0%
2019-05-27 11:32:12,475 *** INFO [robust_forest.py:1162 - __init__()] *** *	Max features: 100.0%
2019-05-27 11:32:12,476 *** INFO [robust_forest.py:1164 - __init__()] *** *	Feature blacklist: set()
2019-05-27 11:32:12,477 *** INFO [robust_forest.py:1165 -

# Standard evaluation metric

The following function is the one used for evaluating the quality of the learned model (either _standard_, _adversarial-boosting_, or _non-interferent_). This is the standard <code>avg_log_loss</code>.

In [3]:
def logistic(x):
    return 1.0/(1.0 + np.exp(-x))

In [4]:
def logit(p):
    return np.log(p/(1-p))

In [5]:
def binarize(preds):
    pos = preds>0
    neg = np.logical_not(pos)
    preds[pos] = 1
    preds[neg] = -1
        
    return preds 

# <code>avg_log_loss</code>

In [6]:
# self-defined eval metric
# f(preds: array, train_data: Dataset) -> name: string, value: array, is_higher_better: bool
def avg_log_loss(preds, train_data):
    
    labels = train_data.get_label()
    losses = np.log(1.0 + np.exp(-preds*labels))
    avg_loss = np.mean(losses)
    
    return 'avg_binary_log_loss', avg_loss, False

In [7]:
def eval_log_loss(preds, test, test_groups=None, svm=False):
        
    labels = test.iloc[:,-1].values
    losses = np.log(1.0 + np.exp(-preds*labels))
    avg_loss = np.mean(losses)
    
    return avg_loss



# Custom evaluation metric

Similarly to what we have done for <code>fobj</code>, <code>feval</code> can be computed from a weighted combination of two evaluation metrics:

-  <code>avg_log_loss</code> (standard, defined above);
-  <code>avg_log_loss_uma</code> (custom, defined below).

# <code>avg_log_loss_uma</code>

This is the binary log loss yet modified to operate on groups of perturbed instances.

In [8]:
# Our custom metric

def binary_log_loss(pred, true_label):

    return np.log(1.0 + np.exp(-pred * true_label))

# self-defined eval metric
# f(preds: array, train_data: Dataset) -> name: string, value: array, is_higher_better: bool
def avg_log_loss_uma(preds, train_data):
    labels = train_data.get_label()
    attack_lens = train_data.get_group()
    
    offset = 0
    max_logloss = []
    avg_max_logloss = 0.0
    
    if attack_lens is not None:
    
        for atk in attack_lens:
            losses = [binary_log_loss(h,t) for h,t in zip(preds[offset:offset+atk], labels[offset:offset+atk])]
            max_logloss.append(max(losses))
            
            offset += atk
        
        avg_max_logloss = np.mean(max_logloss)  
        
    return 'avg_binary_log_loss_under_max_attack', avg_max_logloss, False

In [9]:
def eval_log_loss_uma(preds, test, test_groups=None, svm=False):
    
    lgbm_test = lightgbm.Dataset(data=test.iloc[:,:-1].values, 
                                 label=test.iloc[:,-1].values,
                                 group=test_groups,
                                 free_raw_data=False)
    
    return avg_log_loss_uma(preds,lgbm_test)[1]


# <code>feval=avg_non_interferent_log_loss</code>

Used for measuring the validity of any model (either _standard_, _baseline_, or _non-interferent_). More precisely, <code>avg_non_interferent_log_loss</code> is the weighted sum of the binary log loss and the binary log loss under maximal attack.

In [10]:
# LightGBM takes lambda x,y: avg_weighted_sum_log_loss_log_loss_uma(preds, train_data, alpha=0.5)

def avg_non_interferent_log_loss(preds, train_data, alpha=1.0):
    
    # binary logloss under maximal attack
    _, loss_uma, _    = avg_log_loss_uma(preds, train_data)
    
    # binary logloss (plain)
    # _, loss_plain, _  = avg_log_loss(preds, train_data)
    
    ids = []
    attack_lens = train_data.get_group()
    
    if attack_lens is not None:
        offset=0
        for atk in attack_lens:
            ids += [offset]
            offset += atk      
            
    ids = np.array(ids)
    labels = train_data.get_label()
    losses = np.log(1.0 + np.exp(-preds[ids]*labels[ids]))
    loss_plain = np.mean(losses)

    # combine the above two losses together
    weighted_loss = alpha*loss_uma + (1.0-alpha)*loss_plain

    return 'avg_non_interferent_log_loss [alpha={:.2f}]'.format(alpha), weighted_loss, False

def eval_non_interferent_log_loss(model, test, test_groups=None, svm=False, alpha=1.0):
    
    lgbm_test = lightgbm.Dataset(data=test.iloc[:,:-1].values, 
                                 label=test.iloc[:,-1].values,
                                 group=test_groups,
                                 free_raw_data=False)
    
    return avg_non_interferent_log_loss(model.predict(test.iloc[:,:-1].values), 
                                                  lgbm_test,
                                                  alpha=alpha
                                                 )[1]

# Additional validity measures

In addition to the evaluation metrics defined above (used for training), we also consider the following **4** measures of validity to compare the performance of each learned model:

-  <code>eval_binary_err_rate</code>: This is the traditional binary error rate (1-accuracy);
-  <code>eval_binary_err_rate_uma</code>: This is the binary error rate modified to operate on groups of perturbed instances under maximal attack.
-  <code>eval_roc_auc</code>: This is the classical ROC AUC score;
-  <code>eval_roc_auc_uma</code>: This is the ROC AUC score modified to operate on groups of perturbed instances under maximal attack.

Again, note that those are **not** metrics used at training time (i.e., they do not define any <code>feval</code>), rather they are used to assess the (offline) quality of each learned model.

# <code>eval_binary_err_rate</code>

In [11]:
def eval_binary_err_rate(preds, test_set, test_groups=None, svm=False):
    labels = test_set.iloc[:,-1].values
        
    errs = np.sum(binarize(preds) != labels)
    return errs/len(preds)


# <code>eval_binary_err_rate_uma</code>

In [12]:
def eval_binary_err_rate_uma(preds, test_set, test_groups=None, svm=False):
    preds = binarize(preds)
    labels = test_set.iloc[:,-1].values
    
    offset = 0
    errs = 0

    for g in test_groups:
        predictions_att = preds[offset:offset+g]
        true_label = labels[offset]
        if np.any([p != true_label for p in predictions_att]):
            errs += 1
        offset += g

    return errs/len(test_groups)

# <code>eval_roc_auc</code>

In [13]:
def eval_roc_auc(preds, test_set, test_groups=None, svm=False):
    labels = test_set.iloc[:,-1].values
    return roc_auc_score(y_true=labels, y_score=preds)


# <code>eval_roc_auc_uma</code>

In [14]:
def eval_roc_auc_uma(preds, test_set, test_groups=None, svm=False):
    labels = test_set.iloc[:,-1].values

    offset = 0
    true_labels = []
    worst_predictions = []
    
    for g in test_groups:
        
        true_label = labels[offset]
        true_labels.append(true_label)
        predictions_att = preds[offset:offset+g]
        if true_label == 1:
            worst_predictions.append(np.min(predictions_att))
        else:
            worst_predictions.append(np.max(predictions_att))
    
        offset += g
        
    return roc_auc_score(y_true=true_labels, y_score=worst_predictions)

# <code>eval_specificity</code>

In [15]:
def eval_specificity(preds, test_set, test_groups=None, svm=False):
    preds = binarize(preds)
    labels = test_set.iloc[:,-1].values
    tn, fp, fn, tp = confusion_matrix(y_true=labels, y_pred=preds).ravel()

    return tn/(tn + fp)

# <code>eval_specificity_uma</code>

In [16]:
def eval_specificity_uma(preds, test_set, test_groups=None, svm=False):
    preds = binarize(preds)
    labels = test_set.iloc[:,-1].values

    offset = 0
    true_labels = []
    worst_predictions = []
    
    for g in test_groups:
        true_label = labels[offset]
        true_labels.append(true_label)
        
        predictions_att = preds[offset:offset+g]
        if true_label == 1:
            worst_predictions.append(np.min(predictions_att))
        else:
            worst_predictions.append(np.max(predictions_att))
    
        offset += g
        
    tn, fp, fn, tp = confusion_matrix(y_true=true_labels, y_pred=worst_predictions).ravel()

    return tn/(tn + fp)

# <code>eval_precision</code>

In [17]:
def eval_precision(preds, test_set, test_groups=None, svm=False):
    preds = binarize(preds)
    labels = test_set.iloc[:,-1].values
    tn, fp, fn, tp = confusion_matrix(y_true=labels, y_pred=preds).ravel()

    return tp/(tp + fp)

# <code>eval_precision_uma</code>

In [18]:
def eval_precision_uma(preds, test_set, test_groups=None, svm=False):
    preds = binarize(preds)
    labels = test_set.iloc[:,-1].values

    offset = 0
    true_labels = []
    worst_predictions = []
    
    for g in test_groups:
        true_label = labels[offset]
        true_labels.append(true_label)
        predictions_att = preds[offset:offset+g]
        if true_label == 1:
            worst_predictions.append(np.min(predictions_att))
        else:
            worst_predictions.append(np.max(predictions_att))
    
        offset += g
        
    tn, fp, fn, tp = confusion_matrix(y_true=true_labels, y_pred=worst_predictions).ravel()

    return tp/(tp + fp)

# <code>eval_recall</code>

In [19]:
def eval_recall(preds, test_set, test_groups=None, svm=False):
    preds = binarize(preds)
    labels = test_set.iloc[:,-1].values
    tn, fp, fn, tp = confusion_matrix(y_true=labels, y_pred=preds).ravel()

    return tp/(tp + fn)



# <code>eval_recall_uma</code>

In [20]:
def eval_recall_uma(preds, test_set, test_groups=None, svm=False):
    preds = binarize(preds)
    labels = test_set.iloc[:,-1].values

    offset = 0
    true_labels = []
    worst_predictions = []
    
    for g in test_groups:
        true_label = labels[offset]
        true_labels.append(true_label)
        predictions_att = preds[offset:offset+g]
        if true_label == 1:
            worst_predictions.append(np.min(predictions_att))
        else:
            worst_predictions.append(np.max(predictions_att))
    
        offset += g
        
    tn, fp, fn, tp = confusion_matrix(y_true=true_labels, y_pred=worst_predictions).ravel()

    return tp/(tp + fn)

# <code>eval_f1</code>

In [21]:
def eval_f1(preds, test_set, test_groups=None, svm=False):
    preds = binarize(preds)
    labels = test_set.iloc[:,-1].values    
    return f1_score(y_true=labels, y_pred=preds, average='binary')

# <code>eval_f1_uma</code>

In [22]:
def eval_f1_uma(preds, test_set, test_groups=None, svm=False):
    preds = binarize(preds)
    labels = test_set.iloc[:,-1].values

    offset = 0
    true_labels = []
    worst_predictions = []
    
    for g in test_groups:
        true_label = labels[offset]
        true_labels.append(true_label)
        predictions_att = preds[offset:offset+g]
        if true_label == 1:
            worst_predictions.append(np.min(predictions_att))
        else:
            worst_predictions.append(np.max(predictions_att))
    
        offset += g
        
    return f1_score(y_true=true_labels, y_pred=worst_predictions, average='binary')
    

#     precision_uma = eval_precision_uma(preds, test_set, test_groups=test_groups, svm=svm)
#     recall_uma = eval_recall_uma(preds, test_set, test_groups=test_groups, svm=svm)
    
#     return 2 * (precision_uma * recall_uma)/(precision_uma + recall_uma)


# Evaluate each model w.r.t. _all_ evaluation metrics

In [23]:
def model_predict(model,test_set, svm=False):
    X = test_set.iloc[:,:-1].values 
    if not svm:
        return model.predict(X)
    else:
        return logit(model.predict_proba(X)[:,1])


# Not used yet
def model_worst_predict(preds,test_set):
    labels = test_set.iloc[:,-1].values

    offset = 0
    true_labels = []
    worst_predictions = []
    
    for g in test_groups:
        true_label = labels[offset]
        true_labels.append(true_label)
        predictions_att = preds[offset:offset+g]
        if true_label == 1:
            worst_predictions.append(np.min(predictions_att))
        else:
            worst_predictions.append(np.max(predictions_att))
    
        offset += g
        
        
    return true_labels, worst_predictions

In [24]:
def eval_learned_model(model, eval_metric, test, test_groups=None, svm=False):
    return eval_metric(model, test, test_groups=test_groups, svm=svm)

In [25]:
def eval_learned_models(model, model_type, test, test_groups=None, budget=0):

    eval_metrics = EVAL_METRICS
    d_test = "D_test"
    if test_groups is not None:
        eval_metrics = EVAL_METRICS_UNDER_MAX_ATTACK
        d_test = "D_test_att"
    
    header = ['Model'] + ['Budget'] + [m.__name__.replace('eval_','').replace('_',' ').strip().title() 
                                       for m in EVAL_METRICS]
    df = pd.DataFrame(columns=header)
    first_row = [model_type] + [budget] + [None for m in EVAL_METRICS]
    df.loc[0] = first_row
    
    svm = False
    if model_type == "SVM":
        svm = True

    pred = model_predict(model, test, svm)
        
    for eval_metric in eval_metrics:
        res = eval_learned_model(pred, eval_metric, test, test_groups=test_groups, svm=svm)
        print("{} learning - {} on {} = {:.5f}"
                  .format(model_type, eval_metric.__name__, d_test, res))
        column_metric = eval_metric.__name__
        if eval_metric.__name__.endswith("uma"):
            column_metric = eval_metric.__name__.replace('uma', '')
        df[column_metric.replace('eval_','').replace('_',' ').strip().title()] = res

    print("******************************************************************************************************")
    
    return df

# Load attacked datasets

## Load an attacked dataset with a specific budget

In [26]:
def load_attacked_dataset(budget):
    # load train/valid/test (attacked)
    train_att, valid_att, test_att, _ = load_atk_train_valid_test(TRAINING_FILENAME_ATT.format(budget), 
                                                                  VALIDATION_FILENAME_ATT.format(budget), 
                                                                  TEST_FILENAME_ATT.format(budget))

    test_groups = test_att['instance_id'].value_counts().sort_index().values
    test_att = test_att.iloc[:, 1:]

    valid_groups = valid_att['instance_id'].value_counts().sort_index().values
    valid_att = valid_att.iloc[:, 1:]

    train_groups = train_att['instance_id'].value_counts().sort_index().values
    train_att = train_att.iloc[:, 1:]
    
    return train_att, train_groups, valid_att, valid_groups, test_att, test_groups

## Load _all_ the attacked datasets given a list of budgets

In [27]:
def load_attacked_datasets(budgets):
    att_datasets = {}
    for b in budgets:
        att_datasets[b] = load_attacked_dataset(b)
    
    return att_datasets

# Evaluate all models w.r.t. standard metrics (i.e., attack-free)

In [28]:
def extract_model_name(model_filename):
    model_fileroot = model_filename.split('/')[-1].split('.')[0]
    model_name = model_fileroot.split('_')[0].title()
    training_budget = ''
    budget = model_fileroot.split('_B')[-1].split('_')[0]
    try: 
        int(budget)
        training_budget = ' [train budget={}]'.format(budget)
    except:
        pass
    
    return model_name + training_budget

In [29]:
def load_model(model_file):
    model = None
    try:
        model = lightgbm.Booster(model_file=model_file)
    except:
        print("LightGBM loading exception")
        try:
            with open(model_file, 'rb') as mf:
                model = dill.load(mf)
                print(model)
        except Exception as e:
            print(e)
            print("Dill loading exception")
            pass
    
    return model

In [30]:
def eval_all_models(models_dir, test, model_filenames=None):
    
    if model_filenames is None:
        model_csv = sorted(glob.glob(models_dir + "/*.csv"))
        model_filenames = []

        for m in model_csv:
            model_df = pd.read_csv(m)
            # print(model_df)
            model_filenames.append(model_df.sort_values(by='metric')['filename'].iloc[0])
    
    print ("### Evaluating Models:", model_filenames)
    
    df = pd.concat([eval_learned_models(load_model(mf), 
                                        extract_model_name(mf), 
                                        test) for mf in model_filenames],
                   axis=0,
                   sort=False
                  )
    
    df.reset_index(inplace=True, drop=True)
    
    return df

In [31]:
def eval_all_models_under_attack_budget(models_dir, test, test_groups, budget, model_filenames=None):
    
    #model_filenames = sorted(glob.glob(models_dir + "/*.model"))
    if model_filenames is None:
        model_csv = sorted(glob.glob(models_dir + "/*.csv"))
        model_filenames = []

        for m in model_csv:
            model_df = pd.read_csv(m)
            model_filenames.append(model_df.sort_values(by='metric')['filename'].iloc[0])
    
    print ("### Evaluating Models:", model_filenames)

    df = pd.concat([eval_learned_models(load_model(mf), 
                                        extract_model_name(mf), 
                                        test,
                                        test_groups, 
                                        budget=budget
                                       ) for mf in model_filenames],
                   axis=0,
                   sort=False
                  )
    
    df.reset_index(inplace=True, drop=True)
    
    return df

In [32]:
def eval_all_models_under_attack(models_dir, att_tests, budgets, model_filenames=None):
    
    eval_att_dfs = []

    for b in budgets:
    
        eval_att_dfs.append(
            eval_all_models_under_attack_budget(models_dir, att_tests[b][4], att_tests[b][5], 
                                                b, model_filenames))
        
        
    eval_att_df = functools.reduce(lambda left,right: pd.merge(left,right,on=['Model', 'Budget']), eval_att_dfs)
    eval_att_df = pd.concat(eval_att_dfs, axis=0, sort=False)
    eval_att_df.reset_index(inplace=True, drop=True)
    
    return eval_att_df

# Evaluation metrics

In [33]:
EVAL_METRICS = [eval_log_loss, 
                eval_binary_err_rate,
                eval_specificity,
                eval_precision,
                eval_recall,
                eval_f1,
                eval_roc_auc
               ]

EVAL_METRICS_UNDER_MAX_ATTACK = [eval_log_loss_uma,
                                 eval_binary_err_rate_uma,
                                 eval_specificity_uma,
                                 eval_precision_uma,
                                 eval_recall_uma,
                                 eval_f1_uma,
                                 eval_roc_auc_uma
                                ]

# Census

In [34]:
DATASET_NAME="census"
TRAINING_BUDGETS= [30, 60]

DATASET_DIR="../data/{}".format(DATASET_NAME)
ATK_DIR=DATASET_DIR + "/attacks"
MODELS_DIR="../out/models/{}".format(DATASET_NAME)
OUTPUT_FILENAME="../out/results/{}".format(DATASET_NAME)

TRAINING_FILENAME=DATASET_DIR + "/" + "train.csv.bz2"
TRAINING_FILENAME_ATT=ATK_DIR + "/" + "train_B{}.atks.bz2"

VALIDATION_FILENAME=DATASET_DIR + "/" + "valid.csv.bz2"
VALIDATION_FILENAME_ATT=ATK_DIR + "/" + "valid_B{}.atks.bz2"

TEST_FILENAME=DATASET_DIR + "/" + "test.csv.bz2"
TEST_FILENAME_ATT=ATK_DIR + "/" + "test_B{}.atks.bz2"

In [35]:
# Final Models
# adv_models = ["../out/models/census/adv-boosting_census_B30_T200_S0050_L24_R200.model",
#               "../out/models/census/adv-boosting_census_B60_T200_S0050_L24_R197.model"]
adv_models = ["../out/models/census/adv-boosting_census_B30_T200_S0050_L24_R200.model",
              "../out/models/census/adv-boosting_census_B60_T200_S0050_L24_R200.model"]

# 20 trees
adv_models = ["../out/models/census/adv-boosting_census_B30_T200_S0050_L24_R200.T20.model",
              "../out/models/census/adv-boosting_census_B60_T200_S0050_L24_R200.T20.model"]

# 10K instances
adv_models = ["../out/models/census/adv-boosting_census_B60_T200_S0050_L24_R20.model",
              "../out/models/census/adv-boosting_census_B30_T200_S0050_L24_R41.model"]

gdbt_models = ["../out/models/census/std-gbdt_census_T500_S0050_L24_R207.model",
               "../out/models/census/red-gbdt_census_T500_S0050_L24_R123.model"]
# adding max_depth constraint
gdbt_models = ["../out/models/census/std-gbdt_census_T200_S0050_L24_R195.model",
               "../out/models/census/red-gbdt_census_T200_S0050_L24_R101.model"]

robust_models = ["../out/models/census/robust_census_B60_T100_D8_I20_20_20.model"]
robust_models = ["../out/models/census/robust_census_B0_T100_D8_I20_20_20.model"]

test_models = gdbt_models

In [36]:
# Without attacks
TRAIN, VALID, TEST, _ = load_atk_train_valid_test(TRAINING_FILENAME, VALIDATION_FILENAME, TEST_FILENAME)

eval_std_df = eval_all_models(MODELS_DIR, TEST, test_models)
eval_std_df

Loading pre-processed files...
### Evaluating Models: ['../out/models/census/std-gbdt_census_T200_S0050_L24_R195.model', '../out/models/census/red-gbdt_census_T200_S0050_L24_R101.model']
Std-Gbdt learning - eval_log_loss on D_test = 0.45497
Std-Gbdt learning - eval_binary_err_rate on D_test = 0.13014
Std-Gbdt learning - eval_specificity on D_test = 0.94288
Std-Gbdt learning - eval_precision on D_test = 0.78560
Std-Gbdt learning - eval_recall on D_test = 0.64486
Std-Gbdt learning - eval_f1 on D_test = 0.70830
Std-Gbdt learning - eval_roc_auc on D_test = 0.79387
******************************************************************************************************
Red-Gbdt learning - eval_log_loss on D_test = 0.54586
Red-Gbdt learning - eval_binary_err_rate on D_test = 0.24502
Red-Gbdt learning - eval_specificity on D_test = 1.00000
Red-Gbdt learning - eval_precision on D_test = nan
Red-Gbdt learning - eval_recall on D_test = 0.00000
Red-Gbdt learning - eval_f1 on D_test = 0.00000
Red-Gbd

/home/lucchese/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in long_scalars
  
/home/lucchese/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Model Budget  Log Loss  Binary Err Rate  Specificity  Precision  \
0  Std-Gbdt      0  0.454968         0.130142     0.942882   0.785596   
1  Red-Gbdt      0  0.545863         0.245024     1.000000        NaN   

     Recall        F1   Roc Auc  
0  0.644856  0.708302  0.793869  
1  0.000000  0.000000  0.500000

In [37]:
# %%capture tests

# With attacks
att_datasets = load_attacked_datasets(TRAINING_BUDGETS)

eval_att_df = eval_all_models_under_attack(MODELS_DIR, att_datasets, TRAINING_BUDGETS,
                                           test_models)

overall_df = pd.concat([eval_std_df, eval_att_df], 
                       axis=0, 
                       sort=False)
overall_df.reset_index(inplace=True, drop=True)
overall_df.to_csv(OUTPUT_FILENAME + ".csv", sep=",", index=False)

overall_df

Loading pre-processed files...
Loading pre-processed files...
### Evaluating Models: ['../out/models/census/std-gbdt_census_T200_S0050_L24_R195.model', '../out/models/census/red-gbdt_census_T200_S0050_L24_R101.model']
Std-Gbdt learning - eval_log_loss_uma on D_test_att = 0.51090
Std-Gbdt learning - eval_binary_err_rate_uma on D_test_att = 0.21838
Std-Gbdt learning - eval_specificity_uma on D_test_att = 0.94259
Std-Gbdt learning - eval_precision_uma on D_test_att = 0.61756
Std-Gbdt learning - eval_recall_uma on D_test_att = 0.28565
Std-Gbdt learning - eval_f1_uma on D_test_att = 0.39062
Std-Gbdt learning - eval_roc_auc_uma on D_test_att = 0.61412
******************************************************************************************************
Red-Gbdt learning - eval_log_loss_uma on D_test_att = 0.54586
Red-Gbdt learning - eval_binary_err_rate_uma on D_test_att = 0.24502
Red-Gbdt learning - eval_specificity_uma on D_test_att = 1.00000
Red-Gbdt learning - eval_precision_uma on D_tes

/home/lucchese/.local/lib/python3.6/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in long_scalars
/home/lucchese/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Red-Gbdt learning - eval_roc_auc_uma on D_test_att = 0.50000
******************************************************************************************************
### Evaluating Models: ['../out/models/census/std-gbdt_census_T200_S0050_L24_R195.model', '../out/models/census/red-gbdt_census_T200_S0050_L24_R101.model']
Std-Gbdt learning - eval_log_loss_uma on D_test_att = 0.54900
Std-Gbdt learning - eval_binary_err_rate_uma on D_test_att = 0.24458
Std-Gbdt learning - eval_specificity_uma on D_test_att = 0.91652
Std-Gbdt learning - eval_precision_uma on D_test_att = 0.50175
Std-Gbdt learning - eval_recall_uma on D_test_att = 0.25903
Std-Gbdt learning - eval_f1_uma on D_test_att = 0.34167
Std-Gbdt learning - eval_roc_auc_uma on D_test_att = 0.58777
******************************************************************************************************
Red-Gbdt learning - eval_log_loss_uma on D_test_att = 0.54586
Red-Gbdt learning - eval_binary_err_rate_uma on D_test_att = 0.24502
Red-Gbdt l

Model Budget  Log Loss  Binary Err Rate  Specificity  Precision  \
0  Std-Gbdt      0  0.454968         0.130142     0.942882   0.785596   
1  Red-Gbdt      0  0.545863         0.245024     1.000000        NaN   
2  Std-Gbdt     30  0.510896         0.218377     0.942589   0.617561   
3  Red-Gbdt     30  0.545863         0.245024     1.000000        NaN   
4  Std-Gbdt     60  0.549000         0.244582     0.916520   0.501748   
5  Red-Gbdt     60  0.545863         0.245024     1.000000        NaN   

     Recall        F1   Roc Auc  
0  0.644856  0.708302  0.793869  
1  0.000000  0.000000  0.500000  
2  0.285650  0.390620  0.614120  
3  0.000000  0.000000  0.500000  
4  0.259025  0.341667  0.587773  
5  0.000000  0.000000  0.500000

# Wine

In [39]:
DATASET_NAME="wine"
TRAINING_BUDGETS= [30, 60] 

DATASET_DIR="../data/{}".format(DATASET_NAME)
ATK_DIR=DATASET_DIR + "/attacks"
MODELS_DIR="../out/models/{}".format(DATASET_NAME)
OUTPUT_FILENAME="../out/results/{}".format(DATASET_NAME)

TRAINING_FILENAME=DATASET_DIR + "/" + "train.csv.bz2"
TRAINING_FILENAME_ATT=ATK_DIR + "/" + "train_B{}.atks.bz2"

VALIDATION_FILENAME=DATASET_DIR + "/" + "valid.csv.bz2"
VALIDATION_FILENAME_ATT=ATK_DIR + "/" + "valid_B{}.atks.bz2"

TEST_FILENAME=DATASET_DIR + "/" + "test.csv.bz2"
TEST_FILENAME_ATT=ATK_DIR + "/" + "test_B{}.atks.bz2"

In [40]:
# Final Models
adv_models = ["../out/models/wine/adv-boosting_wine_B30_T200_S0050_L24_R167.model",
              "../out/models/wine/adv-boosting_wine_B60_T200_S0050_L24_R200.model"]

gdbt_models = ["../out/models/wine/std-gbdt_wine_T500_S0050_L24_R497.model",
               "../out/models/wine/red-gbdt_wine_T500_S0050_L24_R497.model"]

gdbt_models = ["../out/models/wine/std-gbdt_wine_T200_S0050_L24_R199.model"]

robust_models = ["../out/models/wine/robust_wine_B30_T50_D8_I20.model",
                 "../out/models/wine/robust_wine_B60_T50_D8_I20.model"]
# robust_models = ["../out/models/wine/robust_wine_B30_T50_D8_I20_1.model",
#                  "../out/models/wine/robust_wine_B60_T50_D8_I20_1.model"]

test_models = gdbt_models

In [41]:
# Without attacks
TRAIN, VALID, TEST, _ = load_atk_train_valid_test(TRAINING_FILENAME, VALIDATION_FILENAME, TEST_FILENAME)

eval_std_df = eval_all_models(MODELS_DIR, TEST, test_models)
eval_std_df

Loading pre-processed files...
### Evaluating Models: ['../out/models/wine/std-gbdt_wine_T200_S0050_L24_R199.model']
Std-Gbdt learning - eval_log_loss on D_test = 0.54035
Std-Gbdt learning - eval_binary_err_rate on D_test = 0.22385
Std-Gbdt learning - eval_specificity on D_test = 0.64990
Std-Gbdt learning - eval_precision on D_test = 0.80716
Std-Gbdt learning - eval_recall on D_test = 0.84933
Std-Gbdt learning - eval_f1 on D_test = 0.82771
Std-Gbdt learning - eval_roc_auc on D_test = 0.74961
******************************************************************************************************


Model Budget  Log Loss  Binary Err Rate  Specificity  Precision  \
0  Std-Gbdt      0  0.540347         0.223846     0.649895   0.807159   

     Recall        F1   Roc Auc  
0  0.849332  0.827709  0.749613

In [42]:
# With attacks
att_datasets = load_attacked_datasets(TRAINING_BUDGETS)

eval_att_df = eval_all_models_under_attack(MODELS_DIR, att_datasets, TRAINING_BUDGETS,
                                           test_models)

overall_df = pd.concat([eval_std_df, eval_att_df], 
                       axis=0, 
                       sort=False)
overall_df.reset_index(inplace=True, drop=True)
overall_df.to_csv(OUTPUT_FILENAME + ".csv", sep=",", index=False)

overall_df

Loading pre-processed files...
Loading pre-processed files...
### Evaluating Models: ['../out/models/wine/std-gbdt_wine_T200_S0050_L24_R199.model']
Std-Gbdt learning - eval_log_loss_uma on D_test_att = 0.62675
Std-Gbdt learning - eval_binary_err_rate_uma on D_test_att = 0.37538
Std-Gbdt learning - eval_specificity_uma on D_test_att = 0.27044
Std-Gbdt learning - eval_precision_uma on D_test_att = 0.66246
Std-Gbdt learning - eval_recall_uma on D_test_att = 0.82989
Std-Gbdt learning - eval_f1_uma on D_test_att = 0.73679
Std-Gbdt learning - eval_roc_auc_uma on D_test_att = 0.55017
******************************************************************************************************
### Evaluating Models: ['../out/models/wine/std-gbdt_wine_T200_S0050_L24_R199.model']
Std-Gbdt learning - eval_log_loss_uma on D_test_att = 0.62972
Std-Gbdt learning - eval_binary_err_rate_uma on D_test_att = 0.37769
Std-Gbdt learning - eval_specificity_uma on D_test_att = 0.26415
Std-Gbdt learning - eval_precis

Model Budget  Log Loss  Binary Err Rate  Specificity  Precision  \
0  Std-Gbdt      0  0.540347         0.223846     0.649895   0.807159   
1  Std-Gbdt     30  0.626749         0.375385     0.270440   0.662464   
2  Std-Gbdt     60  0.629723         0.377692     0.264151   0.660542   

     Recall        F1   Roc Auc  
0  0.849332  0.827709  0.749613  
1  0.829891  0.736785  0.550165  
2  0.829891  0.735595  0.547021

In [ ]:
overall_df

# Prune Robust models

In [33]:
to_be_pruned_models = ["../out/models/census/robust_census_B0_T100_D8_I20_20.tmp"]

for m in to_be_pruned_models:
    prune_trained_model(m, 20)


Saving: ../out/models/census/robust_census_B0_T100_D8_I20_20_20.model


# Prune LGBM models

In [ ]:
def prune_lgbm(in_file, out_file, n):
    model = lightgbm.Booster(model_file=in_file)
    model.save_model(out_file, num_iteration=n)
    print ("saved.")
    
prune_lgbm("../out/models/census/adv-boosting_census_B30_T200_S0050_L24_R200.model",
           "../out/models/census/adv-boosting_census_B30_T200_S0050_L24_R200.T20.model",
           20)
prune_lgbm("../out/models/census/adv-boosting_census_B60_T200_S0050_L24_R200.model",
           "../out/models/census/adv-boosting_census_B60_T200_S0050_L24_R200.T20.model",
           20)

# Feature importance check

In [ ]:
def print_fx_imp(model, colnames):
    fx_uses = model.feature_importance(importance_type='split')
    fx_gain = model.feature_importance(importance_type='gain')

    for i,f in enumerate(np.argsort(fx_gain)[::-1]):
        print ("{:2d} {:20s} {:.3f} {:4d}".format(i, colnames[f], fx_gain[f], fx_uses[f]))

print(" -- GDBT --")    
gbdt = lightgbm.Booster(model_file="../out/models/census/std-gbdt_census_T100_S0050_L24_R100.model")
print(gbdt.num_trees())
print_fx_imp(gbdt, TRAIN.columns)

print(" -- Reduced GDBT --")    
redf = lightgbm.Booster(model_file="../out/models/census/red-gbdt_census_T100_S0050_L24_R98.model")
print(redf.num_trees())
print_fx_imp(redf, TRAIN.drop(columns=["workclass", 
                                       "marital_status", 
                                       "occupation", 
                                       "education_num", 
                                       "hours_per_week", 
                                       "capital_gain"
                                      ]).columns)


print(" -- Adv. Boosting --")    
advb = lightgbm.Booster(model_file="../out/models/census/adv-boosting_census_B30_T100_S0050_L24_R100.model")
print(advb.num_trees())
print_fx_imp(advb, TRAIN.columns)


In [ ]:
bb = 40
eval_learned_models(lightgbm.Booster(model_file="../out/models/wine2/red-gbdt_wine2_T500_S0050_L24_R281.model"), 
                                        extract_model_name("../out/models/wine2/red-gbdt_wine2_T500_S0050_L24_R281.model"), 
                                        att_datasets[bb][4].drop(columns=["alcohol", "residual_sugar", "volatile_acidity"]), 
                                        att_datasets[bb][5], 
                                        budget=bb
                                       ) 